# BRITE Constellation - Carte du Ciel
---
**Tutoriel :** Le script Python suivant utilise des données de l'archive publique des courbes de lumière BRITE-Constellation pour aider les utilisateurs à tracer les coordonnées célestes fournies dans l'en-tête du fichier. Ce qui suit détaillera comment extraire les coordonnées (comme fait dans le tutoriel principal de BRITE Constellation) et les tracer sur une carte du ciel.

**Mission et Instrument :** BRITE-Constellation, les données de ce tutoriel sont prises par le nanosatellite BRITE-Austria

**À propos de la Mission :** La mission BRITE-Constellation se compose de cinq nanosatellites BRIght Target Explorer (BRITE) (trois sont actifs en septembre 2023). Chacun des pays suivants a construit deux satellites : l'Autriche, le Canada et la Pologne. Les six nanosatellites devaient être lancés entre février 2013 et août 2014. Cependant, BRITE-Montreal n'a jamais pu être lancé, donc seulement 5 des 6 étaient opérationnels. L'objectif de la mission est de collecter des données en utilisant la photométrie optique pour en apprendre davantage sur la structure stellaire, l'évolution et les interactions avec l'environnement environnant. Plus d'informations peuvent être trouvées ici : https://brite-constellation.at/.

**Exigences Système :** Accès à Internet, Python Version 3.9 ou supérieure

**Niveau du Tutoriel** : Intermédiaire

---

**Remerciements** : Basé sur des données collectées par la mission satellite BRITE Constellation, conçue, construite, lancée, exploitée et soutenue par l'Agence autrichienne de promotion de la recherche (FFG), l'Université de Vienne, l'Université technique de Graz, l'Agence spatiale canadienne (ASC), l'Institut d'études aérospatiales de l'Université de Toronto (UTIAS), la Fondation pour la science et la technologie polonaises (FNiTP MNiSW), et le Centre national des sciences (NCN).

**Crédits** : Le diagramme d'atlas du ciel utilisé dans ce tutoriel a été créé par Richard Powell du site web [An Atlas of The Universe](http://www.atlasoftheuniverse.com/index.html) et est sous licence Creative Commons Attribution-ShareAlike 2.5. Plus d'informations peuvent être lues [ici](http://www.atlasoftheuniverse.com/copyright.html). L'image du Fond Diffus Cosmologique (CMB) a été prise par le satellite Planck de l'Agence Spatiale Européenne.

**Note : Les coordonnées fournies sont de la zone générale/champ où l'étoile est située et non de l'étoile elle-même.** \
De plus, la précision de ces coordonnées est incertaine.

In [ ]:
#Import libraries
from astropy.coordinates import SkyCoord, Galactic
from astropy.visualization import astropy_mpl_style
from imageio.v2 import imread
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

#### Entrées Utilisateur

In [ ]:
#Name of downloaded .dat file, change this to the file you have saved <str>
# Remember to include the ".dat" at the end
FileName = "HD31237_01-Ori-I-2013_BAb_setup3_APa2s5_DR2.dat"

#Define the desired project <str>
# 'aitoff' is the recommended projection
proj = "aitoff"

#Parameters for plot <int>
X = 13       # Width of the graph
Y = 10       # Height of the graph

### Extraire les Données
Commencez par extraire le nom de l'étoile et les coordonnées célestes de l'en-tête du fichier de courbe de lumière.

**Coordonnées célestes :** Notre système de coordonnées de longitude et latitude est géocentrique (ou géocentrique) et ne peut donc pas être appliqué aux étoiles ou planètes. Au lieu de cela, un système de coordonnées célestes a été développé qui utilise l'*ascension droite (RA)* à la place de la longitude et la *déclinaison (DEC)* à la place de la latitude.

In [ ]:
Path = os.path.realpath(FileName)
DataFrame = pd.read_csv(Path, sep = " ,", on_bad_lines = 'skip', engine = "python", header = None)

# Print Info About Dataset / Print Dataset Header
print ("Header Information")
print (pd.DataFrame(DataFrame.iloc[:37]).to_markdown())

print ()
print ("Data Information")
print (pd.DataFrame(DataFrame.iloc[37:46]).to_markdown())
print ()
# Extract name of star
StarInfo = DataFrame.iloc[6]
StarInfo['new'] = StarInfo.str.split(',').str[0]   
for i in ((np.array(StarInfo['new']))):
    s1 = slice(12, len(i))
    StarName = i[s1]
print ("The name of the star is", StarName)

#Extract RA (Right Acension) and DEC (Declination) of the field (also known as the celestial coordinates)
RA_Info = DataFrame.iloc[8]
DEC_Info = DataFrame.iloc[9]

RA_Info['new'] = RA_Info.str.split('/').str[0] 
for i in (np.array(RA_Info['new'])):
    s1 = slice(13, len(i))
    RA = float(i[s1]) #value of right acension/RA

DEC_Info['new'] = DEC_Info.str.split('/').str[0] 
for i in (np.array(DEC_Info['new'])):
    s1 = slice(13, len(i))
    DEC = float(i[s1]) #value of declination/DEC
print ("The field celestial coordinates are, right acension (RA) is ", RA," and the declination (DEC) is", DEC ," in degrees")

### Tracer les Coordonnées Célestes
D'abord, nous tracerons sur un atlas du ciel.

In [ ]:
## Function for converting right ascenion (RA)
def convert_coords (ra):
    """ Converts the provided coordinate given in radians to be usable on a sky map for plotting. 
        Preferable for x-coordinate/longitude/right ascencion. Y-coordinates/latitude/declination does not
        need to be converted. 

    :param ra: the provided right ascension coordinate in radians
    :type ra: float
    
    :returns ra_convert: the converted coordinates for right ascenion in radians
    :rtype ra_convert: float"""

    # Convert the right ascenion to allow it to wrap around map
    ra_convert = 0
    if ra > (np.pi):
        ra_convert = -1 * ((ra % (np.pi)) - (np.pi)) 
    else:
        ra_convert = -1 * ra
    
    return (ra_convert)

In [ ]:
### Star Atlas Plot ####

star_atlas_url = "http://www.atlasoftheuniverse.com/galchart.gif"

#Initialize plot
img = imread(star_atlas_url)
plt.style.use(astropy_mpl_style)
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
ax.imshow(img, alpha = 0.9)

ax2 = fig.add_subplot(projection = "aitoff")

#Implement object and convert to galactic units
object = SkyCoord(RA, DEC, unit = 'deg')
object_ra = (convert_coords(object.galactic.l.rad))
object_dec = object.galactic.b.rad

#Plot object
ax2.plot(object_ra, object_dec, marker = 'o', markersize = 17, color = 'red', alpha = 0.6)
ax2.text(object_ra, object_dec + 0.1, "Field of " + StarName, color = "red") #Add name of the star

plt.axis("off")
ax.grid(False)
ax.set_title("Visualization of " + StarName + "'s field on a star atlas")
ax.set_xticks([])
ax.set_yticks([])
plt.show()

Maintenant, nous ferons un graphique sur une carte de l'univers avec le CMB en arrière-plan. Nous utiliserons une image créée par le satellite Planck de l'ESA.

In [ ]:
cmb_url = 'https://plancksatellite.org.uk/wp-content/uploads/sites/2/2019/09/PLANCK_FSM_03_cropped_0.png'

#Initialize plot
img = imread(cmb_url)
plt.style.use(astropy_mpl_style)
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
ax.imshow(img, alpha = 0.9)

ax2 = fig.add_subplot(projection = "aitoff")

#Implement object and convert to galactic units
object = SkyCoord(RA, DEC, unit = 'deg')
object_ra = convert_coords(object.galactic.l.rad)
object_dec = object.galactic.b.rad

#Plot object
ax2.plot(object_ra, object_dec, marker = 'o', markersize = 17, color = 'red', alpha = 0.6)
ax2.text(object_ra, object_dec + 0.1, "Field of " + StarName, color = "red")

plt.axis("off")
ax.grid(False)
ax.set_title("Visualization of " + StarName + "'s field on Planck Map")
ax.set_xticks([])
ax.set_yticks([])
plt.show()

**Référence**\
Référence principale pour la carte du ciel. Le tutoriel suivant sur GitHub montre comment tracer des objets dans le ciel en utilisant le package ligo.skymap. Ceci est recommandé pour les utilisateurs Linux seulement.\
https://github.com/ashley-ferreira/WACO/blob/main/WACO_notebook.ipynb

**Lecture Supplémentaire**\
Vous voulez en apprendre davantage sur le tracé de corps célestes ? L'article suivant montre d'autres approches.\
Medium : https://towardsdatascience.com/space-science-with-python-space-maps-747c7d1eaf7f?gi=0f2c97dcdbd9